<font color='green'> 
**Youtube - Aladdin Persson Kanalı - TensorFlow 2.0 Beginner Tutorials serisi**
    
TensorFlow Tutorial 14 - Callbacks with Keras and Writing Custom Callbacks - Aladdin Persson anlattı.
</font>

**Video**: [TensorFlow Tutorial 14 - Callbacks with Keras and Writing Custom Callbacks](https://www.youtube.com/watch?v=WUzLJZCKNu4&list=PLhhyoLH6IjfxVOdVC1P1L5z5azs0XjMsb&index=14)

### İçindekiler

**Loading Dataset**

**Processing the Dataset**

**Creating Model and Training Configuration**

**Callbacks with Keras and Training the Model**

* Callback with Keras - 1
* Callback with Keras - 2

**Writing Custom Callbacks and Training the Model**

* Writing Custom Callbacks - 1
* Writing Custom Callbacks - 2


### <font color="blue"> Giriş</font>

Bu notebookta callbacksi nasıl kullandığımızı ve nasıl custom callbacks oluşturabileceğimizi göreceğiz. Callbacks eğitim veya değerlendirme sırasında modelimizin davranışını özelleştirmenin bir yoludur.

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

### 1. Loading Dataset

In [3]:
(ds_train, ds_test), ds_info = tfds.load(
    "mnist",
    split=["train","test"],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.



Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]


Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


### 2. Preprocessing Dataset


In [4]:
def normalize_img(image, label):
  return tf.cast(image, tf.float32) / 255.0, label

In [6]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 128

ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)

### 3. Creating Model and Training Configuration

In [21]:
model = keras.Sequential(
    [
     keras.Input((28, 28, 1)),
     layers.Conv2D(32, 3, activation="relu"),
     layers.Flatten(),
     layers.Dense(10),
    ]
)

In [9]:
model.compile(
    optimizer=keras.optimizers.Adam(0.01),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

### 4. Callbacks with Keras and Training the Model

#### 4.1. Callback with Keras - 1

Modelin nasıl save ve load edildiğini görmüştük [TensorFlow Tutorial 10 - Saving and Loading Models](https://github.com/eliffkkurt/MyRoadmap/blob/main/TensorFlow/10.%20TensorFlow%20Tutorial%2010%20-%20Saving%20and%20Loading%20Models.ipynb) notebookunda. Save etme işini trainingden sonra yapmıştık `model.fit()`'i çalıştırdıktan sonra. 10 epoch için çalıştırdıysak 10 epochtan sonrasını kaydetmiş oluyoruz bu durumda. Diyelim ki her epochtan sonra kaydetmek istiyoruz veya en iyi modeli kaydetmek istiyoruz. Bunu callback ile yapıyoruz ve modeli fit etmeden önce yapıyoruz.


* `'checkpoint/'` ile file pathi belirledik. 
* `save_best_only=False` en iyi epochu kaydetmek istiyorsak bunu True yapıyoruz.

Kerasın farklı callbackleri var. Dökümentasyonundan bakabilirsin.

In [10]:
save_callback = keras.callbacks.ModelCheckpoint(
    'checkpoint/', 
    save_weights_only=True, 
    monitor='accuracy', # validation setin de varsa burada onu belirtmen gerekir.
    save_best_only=False, 
)

Yazdığımız callbacki `model.fit()` içerisine `call_backs=[save_callback]` olarak ekledik.

In [13]:
model.fit(
    ds_train,
    epochs=10,
    verbose=2,
    callbacks=[save_callback]
)

Epoch 1/10
469/469 - 16s - loss: 0.0114 - accuracy: 0.9963
Epoch 2/10
469/469 - 16s - loss: 0.0076 - accuracy: 0.9976
Epoch 3/10
469/469 - 16s - loss: 0.0091 - accuracy: 0.9971
Epoch 4/10
469/469 - 16s - loss: 0.0075 - accuracy: 0.9973
Epoch 5/10
469/469 - 16s - loss: 0.0108 - accuracy: 0.9967
Epoch 6/10
469/469 - 16s - loss: 0.0102 - accuracy: 0.9969
Epoch 7/10
469/469 - 16s - loss: 0.0069 - accuracy: 0.9976
Epoch 8/10
469/469 - 16s - loss: 0.0071 - accuracy: 0.9979
Epoch 9/10
469/469 - 16s - loss: 0.0060 - accuracy: 0.9984
Epoch 10/10
469/469 - 16s - loss: 0.0108 - accuracy: 0.9973


#### 4.2. Callback with Keras - 2

Farklı bir callback daha ekleyeceğiz. Diyelim ki learning rate planlayıcısı olsun bu callbackin. Bizim learning rate'imiz başlangıçta 0.01 idi. Epoch ilerlerken learning rate'i değiştireceğiz. 

In [14]:
def scheduler(epoch, lr):
  if epoch<2:
    return lr

  else:
    return lr * 0.99 # Yani her epochta %1 azaltmış oluyoruz. 

In [15]:
lr_scheduler = keras.callbacks.LearningRateScheduler(scheduler, verbose=1)

Bunu callbacks listemize ekliyoruz `model.fit()` içerisinde.

In [16]:
model.fit(
    ds_train,
    epochs=10,
    verbose=2,
    callbacks=[save_callback, lr_scheduler]
)

Epoch 1/10

Epoch 00001: LearningRateScheduler setting learning rate to 0.009999999776482582.
469/469 - 16s - loss: 0.0129 - accuracy: 0.9971
Epoch 2/10

Epoch 00002: LearningRateScheduler setting learning rate to 0.009999999776482582.
469/469 - 16s - loss: 0.0051 - accuracy: 0.9987
Epoch 3/10

Epoch 00003: LearningRateScheduler setting learning rate to 0.009899999778717757.
469/469 - 16s - loss: 0.0023 - accuracy: 0.9993
Epoch 4/10

Epoch 00004: LearningRateScheduler setting learning rate to 0.009800999946892262.
469/469 - 16s - loss: 0.0035 - accuracy: 0.9990
Epoch 5/10

Epoch 00005: LearningRateScheduler setting learning rate to 0.009702990353107453.
469/469 - 16s - loss: 0.0112 - accuracy: 0.9973
Epoch 6/10

Epoch 00006: LearningRateScheduler setting learning rate to 0.009605960855260491.
469/469 - 16s - loss: 0.0072 - accuracy: 0.9981
Epoch 7/10

Epoch 00007: LearningRateScheduler setting learning rate to 0.00950990131124854.
469/469 - 16s - loss: 0.0068 - accuracy: 0.9982
Epoch 8

### 5. Writing Custom Callbacks and Training the Model

Kendi callbacklerimizi yazabilmek için Keras'ın [Writing your own custom callbacks](https://www.tensorflow.org/guide/keras/custom_callback) dokümanında yer alan `class CustomCallback(keras.callbacks.Callback):` classından yararlandık. Hangi aşamada customize etmek istiyorsak o aşamadaki fonksiyon için bir şey yazıyoruz. Biz sadece `def on_epoch_end(self, epoch, logs=None)` fonksiyonunu kullandık.

#### 5.1. Writing Custom Callbacks - 1

In [17]:
class CustomCallback(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    print(logs.keys()) # tüm epochları bastırıyor.

In [18]:
model.fit(
    ds_train,
    epochs=10,
    verbose=2,
    callbacks=[save_callback, lr_scheduler, CustomCallback()]
)

Epoch 1/10

Epoch 00001: LearningRateScheduler setting learning rate to 0.009227447211742401.
469/469 - 16s - loss: 0.0026 - accuracy: 0.9992
dict_keys(['loss', 'accuracy', 'lr'])
Epoch 2/10

Epoch 00002: LearningRateScheduler setting learning rate to 0.009227447211742401.
469/469 - 17s - loss: 0.0062 - accuracy: 0.9984
dict_keys(['loss', 'accuracy', 'lr'])
Epoch 3/10

Epoch 00003: LearningRateScheduler setting learning rate to 0.009135172739624976.
469/469 - 16s - loss: 0.0034 - accuracy: 0.9991
dict_keys(['loss', 'accuracy', 'lr'])
Epoch 4/10

Epoch 00004: LearningRateScheduler setting learning rate to 0.009043820975348353.
469/469 - 16s - loss: 0.0038 - accuracy: 0.9991
dict_keys(['loss', 'accuracy', 'lr'])
Epoch 5/10

Epoch 00005: LearningRateScheduler setting learning rate to 0.008953382922336458.
469/469 - 17s - loss: 0.0072 - accuracy: 0.9986
dict_keys(['loss', 'accuracy', 'lr'])
Epoch 6/10

Epoch 00006: LearningRateScheduler setting learning rate to 0.008863849360495805.
469/46

Yazdığımız fonksiyonla sadece loss, accuracy ve learning rate'i görmüş olduk. 

#### 5.2. Writing Custom Callbacks - 2

Diyelim ki accuracy %90'ın üzerindeyse trainingi durdurmak istiyoruz. 

In [19]:
class CustomCallback(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    if logs.get("accuracy") > 0.90:  # eğer validation setimiz olsaydı buraya "val_accuracy" yazardık.
      print("Accuracy over %90, quitting training")
      self.model.stop_training = True 

Burada epoch sonunda kontrol et dedik ama batchten sonra da kontrol etmesini isteyebilirdik. O zaman on_batch_end fonksiyonunu kullanmamız gerekirdi.

In [20]:
model.fit(
    ds_train,
    epochs=10,
    verbose=2,
    callbacks=[save_callback, lr_scheduler, CustomCallback()]
)

Epoch 1/10

Epoch 00001: LearningRateScheduler setting learning rate to 0.008514578454196453.
469/469 - 17s - loss: 0.0036 - accuracy: 0.9993
Accuracy over %90, quitting training
